In [ ]:
### Neural Network models developed by
### Ezequiel Piedras, 2022
### Lab4 skeleton and helper functions provided in Reinforcement Learning course were used

import gymenv_v2
from gymenv_v2 import make_multiple_env
from gymenv_v2 import timelimit_wrapper, GurobiOriginalEnv
import numpy as np
import pandas as pd
import tensorflow as tf
import importlib
import matplotlib
from matplotlib import pyplot as plt
import wandb

import multiprocessing
import time

In [ ]:
class Policy(object):

    def __init__(self, state_dim, lr, hidden_size):

        # Using Keras Functional API to build my custom model
        input1 = tf.keras.Input(shape=(state_dim,1))
        input2 = tf.keras.Input(shape=(state_dim,1))

        lstm1 = tf.keras.layers.LSTM(8)(input1)
        lstm2 = tf.keras.layers.LSTM(8)(input2)
        dense1a = tf.keras.layers.Dense(hidden_size, activation = "tanh")(lstm1)
        dense2a = tf.keras.layers.Dense(hidden_size)(dense1a)
        dense1b = tf.keras.layers.Dense(hidden_size, activation = "tanh")(lstm2)
        dense2b = tf.keras.layers.Dense(hidden_size)(dense1b)

        output = tf.reduce_mean(tf.tensordot(dense2a,tf.transpose(dense2b), axes = 1), axis = 0)

        self.model = tf.keras.Model(inputs = [input1,input2], outputs = output)

        # DEFINE THE OPTIMIZER
        self.optimizer = tf.keras.optimizers.Adam(learning_rate = lr)

        # RECORD HYPER-PARAMS
        self.state_dim = state_dim

    def call(self, state):

        # Unwrap state tuple
        A = state[0]
        b = state[1]
        # Normalize constraints
        A = np.divide(A.transpose(), b).transpose()
        A = np.divide(A,(np.max(A,axis=0)-np.min(A,axis=0))) - np.min(A,axis=0)
        A = tf.expand_dims(A, axis = 2)
        A = tf.cast(A, tf.double)

        D = state[3]
        e = state[4]
        # Normalize cuts
        D = np.divide(D.transpose(), e).transpose()
        D = np.divide(D, (np.max(D, axis=0) - np.min(D, axis=0))) - np.min(D, axis=0)
        D = tf.expand_dims(D, axis = 2)
        D = tf.cast(D, tf.double)

        # Call model
        scores = self.model([A,D])
        return scores


    def compute_prob(self, state):

        prob = tf.cast(tf.nn.softmax(self.call(state), axis = -1), tf.double)  # probabilities computed as softmax, sum to 1
        return prob.numpy()

    def train(self, states, actions, Es):

        with tf.GradientTape() as tape:
            # COMPUTE probability vector pi(s) for all s in states
            total_loss = 0
            N = len(states)

            for i in range(N):

                prob = tf.cast(tf.nn.softmax(self.call(states[i]), axis=-1), tf.double)
                action_onehot = tf.cast(tf.one_hot(actions[i], len(states[i][-1])), tf.double)
                prob_selected = tf.reduce_sum(prob * action_onehot, axis=-1)

                # FOR ROBUSTNESS
                prob_selected += 1e-8

                total_loss += -1 * Es[i] * tf.math.log(prob_selected)

            # BACKWARD PASS
            total_loss_2 = total_loss / N
            gradients = tape.gradient(total_loss_2, self.model.trainable_variables)

            # UPDATE
            self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

        return total_loss_2.numpy()


# Helper function
def discounted_rewards(r, gamma):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_sum = 0
    for i in reversed(range(0, len(r))):
        discounted_r[i] = running_sum * gamma + r[i]
        running_sum = discounted_r[i]
    return list(discounted_r)

In [ ]:
## Helper function for testing
def make_gurobi_env(load_dir, idx, timelimit):
	print('loading training instances, dir {} idx {}'.format(load_dir, idx))
	A = np.load('{}/A_{}.npy'.format(load_dir, idx))
	b = np.load('{}/b_{}.npy'.format(load_dir, idx))
	c = np.load('{}/c_{}.npy'.format(load_dir, idx))
	env = timelimit_wrapper(GurobiOriginalEnv(A, b, c, solution=None, reward_type='obj'), timelimit)
	return env

In [ ]:

## Training data setup
# Setup: You may generate your own instances on which you train the cutting agent.
custom_config = {
    "load_dir"        : 'instances/randomip_n60_m60',   # this is the location of the randomly generated instances (you may specify a different directory)
    "idx_list"        : list(range(20)),                # take the first 20 instances from the directory
    "timelimit"       : 50,                             # the maximum horizon length is 50
    "reward_type"     : 'obj'                           # DO NOT CHANGE reward_type
}

# Easy Setup: Use the following environment settings. We will evaluate your agent with the same easy config below:
easy_config = {
    "load_dir"        : 'instances/train_10_n60_m60',
    "idx_list"        : list(range(10)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}

# Hard Setup: Use the following environment settings. We will evaluate your agent with the same hard config below:
hard_config = {
    "load_dir"        : 'instances/train_100_n60_m60',
    "idx_list"        : list(range(99)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}

In [ ]:
### Choose argument (hyper-parameter) for Policy function

argument = 8

### Choose easy or hard setup
dif = "easy" # easy OR hard
# create env
if dif == "easy":
    env = make_multiple_env(**easy_config)
else:
    env = make_multiple_env(**hard_config)

### TRAINING
run = wandb.init(project="finalproject", entity="ezequiel-piedras",
                 tags=["training-{}".format(dif)],
                 name="{dif}-{a:.3f}".format(dif=dif,a=argument))

# Initialize parameters
actor_lr = 1e-2  # learning rate for actor
numtrajs = 10  # num of trajectories from the current policy to collect in each iteration
iterations = 50  # total num of iterations
gamma = .1  # discount
sigma = 10.
state_dim = 60

# Instantiate actor model
actor = Policy(state_dim, actor_lr, argument)

# Training reward history
r_history = []

for ite in range(iterations):

    # For recording trajectories played in this iteration
    OBSERVED_STATES = []
    ACTIONS = []
    MC_VALUES = []

    # Record avg. iteration reward
    ravgit = np.zeros((numtrajs))

    for traj in range(numtrajs):

        observed_states = []
        actions = []
        rewards = []

        s = env.reset()  # samples a random instance every time env.reset() is called
        done = False

        t = 0
        repisode = 0

        while not done:
            # Run policy
            action_space_size = s[-1].size  # size of action space changes every time
            prob = actor.compute_prob(s)
            prob = prob.flatten()
            prob /= np.sum(prob)
            action = np.random.choice(action_space_size, p=prob, size=1)
            new_s, r, done, _ = env.step(action)

            # Save the move
            observed_states.append(s)
            actions.append(action[0])
            rewards.append(r)

            # Update trackers
            s = new_s
            t += 1
            repisode += r

        # Record trajectory
        V_hats = discounted_rewards(np.array(rewards), gamma)
        OBSERVED_STATES.extend(observed_states)
        ACTIONS.extend(actions)
        MC_VALUES.extend(V_hats)

        # Log training reward
        ravgit[traj] = repisode

    # Update policy model
    ES = MC_VALUES + np.random.normal(0, 1, len(MC_VALUES)) / sigma
    progr = actor.train(OBSERVED_STATES, ACTIONS, ES)
    # Print training progress
    ravgit = np.mean(ravgit)
    r_history.append(ravgit)
    wandb.log({"Training reward ({} config)".format(dif): ravgit})
    print("Argu: {argu:2.3f}    It: {ite:2.0f}   Loss: {progr: .3f}   R: {repisode:.3f}" \
          .format(argu=argument, ite=ite, progr=progr, repisode=ravgit))

r_history = pd.DataFrame(r_history)
r_history['MAvg'] = r_history.rolling(5, min_periods=5).mean()
#r_history.plot(figsize=(10, 5))


### Also test the policy function in the random-generated ILPs
r_test = np.zeros((10))
for env_i in range(10):
    env = make_gurobi_env('instances/randomip_n60_m60', env_i, 100)
    s = env.reset()
    d = False
    t = 0
    rtestit = 0.
    while not d:
        # Run policy
        action_space_size = s[-1].size  # size of action space changes every time
        prob = actor.compute_prob(s)
        prob = prob.flatten()
        prob /= np.sum(prob)
        action = np.random.choice(action_space_size, p=prob, size=1)
        s, r, d, _ = env.step(action)
        #print('step', t, 'reward', r, 'action space size', s[-1].size)
        rtestit += r
        t += 1
    r_test[env_i] = rtestit
    wandb.log({"Test reward ({} config)".format(dif): rtestit})

print("Argu: {argu:2.3f}. Average reward on test set: {avg:.3f}".\
      format(argu=argument,avg=np.mean(r_test)))
wandb.finish()

r_history